In [1]:
import pandas as pd
import numpy as np
import math

from scipy.stats import chi2
from statsmodels.stats.contingency_tables import StratifiedTable, Table2x2
from tabulate import tabulate
from scipy.stats import norm

df = pd.read_csv('relativerisk.csv')
df.head()

,ageclass,married,healthy
0,4.0,1,0
1,3.0,0,0
2,2.0,1,0
3,1.0,1,0
4,4.0,1,0


In [2]:
df['married'] = df['married'].replace({0: 'unmarried', 1: 'married'})
df['healthy'] = df['healthy'].replace({0: 'unhealthy', 1: 'healthy'})

crosstab = pd.crosstab(df['married'], df['healthy'], margins=True)
print("Crosstab between married and healthy:")
print(tabulate(crosstab, headers='keys', tablefmt='grid'))

crosstab_perc = crosstab.div(crosstab['All'], axis=0) * 100
print("\nPercentages (%):")
print(tabulate(crosstab_perc, headers='keys', tablefmt='grid'))

Crosstab between married and healthy:
+-----------+-----------+-------------+-------+
| married   |   healthy |   unhealthy |   All |
+===========+===========+=============+=======+
| married   |      1537 |         167 |  1704 |
+-----------+-----------+-------------+-------+
| unmarried |      1104 |         192 |  1296 |
+-----------+-----------+-------------+-------+
| All       |      2641 |         359 |  3000 |
+-----------+-----------+-------------+-------+

Percentages (%):
+-----------+-----------+-------------+-------+
| married   |   healthy |   unhealthy |   All |
+===========+===========+=============+=======+
| married   |   90.1995 |     9.80047 |   100 |
+-----------+-----------+-------------+-------+
| unmarried |   85.1852 |    14.8148  |   100 |
+-----------+-----------+-------------+-------+
| All       |   88.0333 |    11.9667  |   100 |
+-----------+-----------+-------------+-------+


In [3]:
unmarried_unhealthy = crosstab.loc["unmarried", "unhealthy"]
unmarried_healthy = crosstab.loc["unmarried", "healthy"]
married_unhealthy = crosstab.loc["married", "unhealthy"]
married_healthy = crosstab.loc["married", "healthy"]

table = np.array([
    [unmarried_unhealthy, unmarried_healthy],
    [married_unhealthy, married_healthy]
])

table2x2 = Table2x2(table)

oddsratio = table2x2.oddsratio
oddsratio_confint = table2x2.oddsratio_confint()
print(f"\nOdds Ratio (no/yes): {oddsratio}")
print(f"95% Confidence Interval for Odds Ratio:")  
print(f"lower bound: {oddsratio_confint[0]}")
print(f"upper bound: {oddsratio_confint[1]}")


Odds Ratio (no/yes): 1.6006248372819578
95% Confidence Interval for Odds Ratio:
lower bound: 1.2828176176769495
upper bound: 1.9971661087438233


In [4]:
unmarried_total = unmarried_unhealthy + unmarried_healthy
married_total = married_unhealthy + married_healthy

prob_unmarried_unhealthy = unmarried_unhealthy / unmarried_total
prob_married_unhealthy = married_unhealthy / married_total

relative_risk_unhealthy = prob_unmarried_unhealthy / prob_married_unhealthy
print(f"\nRelative Risk (unhealthy): {relative_risk_unhealthy}")

se_relative_risk_unhealthy = math.sqrt(
    (1 - prob_unmarried_unhealthy) / (unmarried_total * prob_unmarried_unhealthy) +
    (1 - prob_married_unhealthy) / (married_total * prob_married_unhealthy)
)

ci_lower_unhealthy = relative_risk_unhealthy * math.exp(-1.96 * se_relative_risk_unhealthy)
ci_upper_unhealthy = relative_risk_unhealthy * math.exp(1.96 * se_relative_risk_unhealthy)

print(f"95% Confidence Interval for Relative Risk (healthy = no):") 
print(f"lower bound: {ci_lower_unhealthy}")
print(f"upper bound: {ci_upper_unhealthy}")


Relative Risk (unhealthy): 1.511643379906853
95% Confidence Interval for Relative Risk (healthy = no):
lower bound: 1.2445729778426209
upper bound: 1.8360238802365885


In [5]:
prob_unmarried_healthy = unmarried_healthy / unmarried_total
prob_married_healthy = married_healthy / married_total

relative_risk_healthy = prob_unmarried_healthy / prob_married_healthy
print(f"\nRelative Risk (healthy): {relative_risk_healthy}")

se_relative_risk_healthy = math.sqrt(
    (1 - prob_unmarried_healthy) / (unmarried_total * prob_unmarried_healthy) +
    (1 - prob_married_healthy) / (married_total * prob_married_healthy)
)

ci_lower_healthy = relative_risk_healthy * math.exp(-1.96 * se_relative_risk_healthy)
ci_upper_healthy = relative_risk_healthy * math.exp(1.96 * se_relative_risk_healthy)

print(f"95% Confidence Interval for Relative Risk (healthy = yes):")
print(f"lower bound: {ci_lower_healthy}")
print(f"upper bound: {ci_upper_healthy}")


Relative Risk (healthy): 0.9444082989951565
95% Confidence Interval for Relative Risk (healthy = yes):
lower bound: 0.9187205791070823
upper bound: 0.9708142557095893


In [6]:
ageclass_mapping = {1.0: '30-40', 2: '40-50', 3: '50-60', 4: '60-70'}

ageclasses = sorted(df['ageclass'].unique())
tables = {}

for age in ageclasses:
    df_age = df[df['ageclass'] == age]
    
    crosstab_count = pd.crosstab(df_age['married'], df_age['healthy'], margins=True, margins_name="Total")
    crosstab_perc = crosstab_count.div(crosstab_count['Total'], axis=0) * 100
    
    combined_df = pd.DataFrame()
    combined_df['Unhealthy (Count)'] = crosstab_count['unhealthy']
    combined_df['Healthy (Count)'] = crosstab_count['healthy']
    combined_df['Total (Count)'] = crosstab_count['Total']
    combined_df['Unhealthy (% within married)'] = crosstab_perc['unhealthy']
    combined_df['Healthy (% within married)'] = crosstab_perc['healthy']
    combined_df['Total (%)'] = crosstab_perc['Total']
    
    age_label = ageclass_mapping.get(age, age)
    tables[age_label] = combined_df

for age, table in tables.items():
    print(f"\nAgeclass {age}")
    print(tabulate(table, headers='keys', tablefmt='grid'))



Ageclass 30-40
+-----------+---------------------+-------------------+-----------------+--------------------------------+------------------------------+-------------+
| married   |   Unhealthy (Count) |   Healthy (Count) |   Total (Count) |   Unhealthy (% within married) |   Healthy (% within married) |   Total (%) |
+===========+=====================+===================+=================+================================+==============================+=============+
| married   |                  53 |               327 |             380 |                        13.9474 |                      86.0526 |         100 |
+-----------+---------------------+-------------------+-----------------+--------------------------------+------------------------------+-------------+
| unmarried |                  52 |               138 |             190 |                        27.3684 |                      72.6316 |         100 |
+-----------+---------------------+-------------------+-----------------

In [7]:
z_score = norm.ppf(0.975) 
results_with_ci = []

for age in ageclasses:
    df_age = df[df['ageclass'] == age]
    age_label = ageclass_mapping.get(age, age)
    
    crosstab = pd.crosstab(df_age['married'], df_age['healthy'])

    if 'unmarried' in crosstab.index and 'married' in crosstab.index and 'unhealthy' in crosstab.columns and 'healthy' in crosstab.columns:
        table = np.array([
            [crosstab.loc['unmarried', 'unhealthy'], crosstab.loc['unmarried', 'healthy']],
            [crosstab.loc['married', 'unhealthy'], crosstab.loc['married', 'healthy']]
        ])

        table2x2 = Table2x2(table)
        odds_ratio = table2x2.oddsratio
        or_confint = table2x2.oddsratio_confint()

        total_unmarried = table[0].sum()
        total_married = table[1].sum()

        prob_unmarried_unhealthy = table[0, 0] / total_unmarried if total_unmarried > 0 else np.nan
        prob_married_unhealthy = table[1, 0] / total_married if total_married > 0 else np.nan

        prob_unmarried_healthy = table[0, 1] / total_unmarried if total_unmarried > 0 else np.nan
        prob_married_healthy = table[1, 1] / total_married if total_married > 0 else np.nan

        relative_risk_unhealthy = prob_unmarried_unhealthy / prob_married_unhealthy if prob_married_unhealthy > 0 else np.nan
        relative_risk_healthy = prob_unmarried_healthy / prob_married_healthy if prob_married_healthy > 0 else np.nan

        if prob_unmarried_unhealthy > 0 and prob_married_unhealthy > 0:
            se_relative_risk_unhealthy = math.sqrt(
                (1 - prob_unmarried_unhealthy) / (total_unmarried * prob_unmarried_unhealthy) +
                (1 - prob_married_unhealthy) / (total_married * prob_married_unhealthy)
            )
            ci_lower_unhealthy = relative_risk_unhealthy * math.exp(-z_score * se_relative_risk_unhealthy)
            ci_upper_unhealthy = relative_risk_unhealthy * math.exp(z_score * se_relative_risk_unhealthy)
        else:
            ci_lower_unhealthy = ci_upper_unhealthy = np.nan

        if prob_unmarried_healthy > 0 and prob_married_healthy > 0:
            se_relative_risk_healthy = math.sqrt(
                (1 - prob_unmarried_healthy) / (total_unmarried * prob_unmarried_healthy) +
                (1 - prob_married_healthy) / (total_married * prob_married_healthy)
            )
            ci_lower_healthy = relative_risk_healthy * math.exp(-z_score * se_relative_risk_healthy)
            ci_upper_healthy = relative_risk_healthy * math.exp(z_score * se_relative_risk_healthy)
        else:
            ci_lower_healthy = ci_upper_healthy = np.nan

        valid_cases = df_age.shape[0]
        
        results_with_ci.append({
            'Ageclass': age_label,
            'Odds Ratio for married (no/yes)': odds_ratio,
            'Odds Ratio CI Lower': or_confint[0],
            'Odds Ratio CI Upper': or_confint[1],
            'For cohort healthy=no': relative_risk_unhealthy,
            '95% CI Lower (healthy=no)': ci_lower_unhealthy,
            '95% CI Upper (healthy=no)': ci_upper_unhealthy,
            'For cohort healthy=yes': relative_risk_healthy,
            '95% CI Lower (healthy=yes)': ci_lower_healthy,
            '95% CI Upper (healthy=yes)': ci_upper_healthy,
            'N of valid cases': valid_cases
        })

results_with_ci_df = pd.DataFrame(results_with_ci)
results_with_ci_df

,Ageclass,Odds Ratio for married (no/yes),Odds Ratio CI Lower,Odds Ratio CI Upper,For cohort healthy=no,95% CI Lower (healthy=no),95% CI Upper (healthy=no),For cohort healthy=yes,95% CI Lower (healthy=yes),95% CI Upper (healthy=yes),N of valid cases
0,30-40,2.324856,1.510517,3.578217,1.962264,1.395805,2.758609,0.844037,0.766614,0.929279,570
1,40-50,1.734947,1.209090,2.489511,1.614493,1.180420,2.208187,0.930572,0.885666,0.977754,1081
2,50-60,2.351771,1.253768,4.411361,2.186489,1.227015,3.896230,0.929720,0.879486,0.982823,533
3,60-70,1.145278,0.702901,1.866071,1.131579,0.724928,1.766343,0.988038,0.946473,1.031429,816


In [8]:
odds_ratios = []
variances = []
tables = []

for age in ageclasses:
    df_age = df[df['ageclass'] == age]
    crosstab = pd.crosstab(df_age['married'], df_age['healthy'])
    
    table = np.array([
        [crosstab.loc['unmarried', 'unhealthy'], crosstab.loc['unmarried', 'healthy']],
        [crosstab.loc['married', 'unhealthy'], crosstab.loc['married', 'healthy']]
    ])
    tables.append(table)
    
    table2x2 = Table2x2(table)
    or_k = table2x2.oddsratio
    log_or_k = math.log(or_k)
    var_k = 1 / table[0, 0] + 1 / table[0, 1] + 1 / table[1, 0] + 1 / table[1, 1]
    
    odds_ratios.append(log_or_k)
    variances.append(var_k)

pooled_or = sum(odds_ratios[i] / variances[i] for i in range(len(odds_ratios))) / sum(1 / variances[i] for i in range(len(variances)))
pooled_or = math.exp(pooled_or)  

breslow_day_stat = sum((odds_ratios[i] - math.log(pooled_or)) ** 2 / variances[i] for i in range(len(odds_ratios)))

df_breslow_day = len(ageclasses) - 1

print(f"Breslow-Day test statistic: {breslow_day_stat}")
print(f"Degrees of freedom: {df_breslow_day}")

p_value = 1 - chi2.cdf(breslow_day_stat, df_breslow_day)
print(f"P-value for Breslow-Day test: {p_value}")

Breslow-Day test statistic: 5.3785690463385345
Degrees of freedom: 3
P-value for Breslow-Day test: 0.14608464522198905


In [9]:
avg_variance = sum(variances) / len(variances)
tarone_stat = sum((odds_ratios[i] - math.log(pooled_or)) ** 2 / avg_variance for i in range(len(odds_ratios)))

df_tarone_stat = len(ageclasses) - 1
p_value_tarone = 1 - chi2.cdf(tarone_stat, df_tarone_stat)

print(f"\nTarone's test statistic: {tarone_stat}")
print(f"Degrees of freedom: {df_tarone_stat}")
print(f"Asymp. Sig. (2-sided) for Tarone's test: {p_value_tarone}")



Tarone's test statistic: 5.551745374432485
Degrees of freedom: 3
Asymp. Sig. (2-sided) for Tarone's test: 0.13557622129353464


In [10]:
stratified_table = StratifiedTable(tables)

mantel_haenszel_result = stratified_table.test_null_odds()
mantel_haenszel_stat = mantel_haenszel_result.statistic
mantel_haenszel_p = mantel_haenszel_result.pvalue
mantel_haenszel_df = 1

print("\nCochran-Mantel-Haenszel Chi-Squared Test:")
print(f"CMH Chi-squared: {mantel_haenszel_stat}")
print(f"Degrees of freedom: {mantel_haenszel_df}")
print(f"Asymp. Sig. (2-sided) for CMH Test: {mantel_haenszel_p}")

mh_or = stratified_table.oddsratio_pooled
mh_or_confint = stratified_table.oddsratio_pooled_confint()
print(f"\nMantel-Haenszel common Odds Ratio: {mh_or}")
print(f"95% Confidence Interval: {mh_or_confint}")



Cochran-Mantel-Haenszel Chi-Squared Test:
CMH Chi-squared: 26.090522527525195
Degrees of freedom: 1
Asymp. Sig. (2-sided) for CMH Test: 3.257791911792651e-07

Mantel-Haenszel common Odds Ratio: 1.780649273613681
95% Confidence Interval: (1.4217473900860704, 2.230151331896575)


In [11]:
common_odds_ratio = stratified_table.oddsratio_pooled
ln_common_odds_ratio = math.log(common_odds_ratio)

variance_ln_or = sum(1 / (table[0, 0]) + 1 / (table[0, 1]) + 1 / (table[1, 0]) + 1 / (table[1, 1]) for table in tables)
std_error_ln_or = math.sqrt(variance_ln_or)

ci_lower_ln_or = ln_common_odds_ratio - z_score * std_error_ln_or
ci_upper_ln_or = ln_common_odds_ratio + z_score * std_error_ln_or
ci_lower_or = math.exp(ci_lower_ln_or)
ci_upper_or = math.exp(ci_upper_ln_or)

z_value = ln_common_odds_ratio / std_error_ln_or
p_value = 2 * (1 - norm.cdf(abs(z_value)))

print("Mantel-Haenszel common odds ratio estimate:")
print(f"Estimate (Common Odds Ratio): {common_odds_ratio}")
print(f"ln(Estimate): {ln_common_odds_ratio}")
print(f"Std. Error of ln(Estimate): {std_error_ln_or}")
print(f"Asymp. Sig. (2-sided): {p_value}")
print("\nAsymp. 95% confidence interval:")
print(f"Common odds ratio: Lower bound: {ci_lower_or}, Upper bound: {ci_upper_or}")
print(f"ln(Common odds ratio): Lower bound: {ci_lower_ln_or}, Upper bound: {ci_upper_ln_or}")

Mantel-Haenszel common odds ratio estimate:
Estimate (Common Odds Ratio): 1.780649273613681
ln(Estimate): 0.5769780582521951
Std. Error of ln(Estimate): 0.49737970296370654
Asymp. Sig. (2-sided): 0.2460343971681589

Asymp. 95% confidence interval:
Common odds ratio: Lower bound: 0.6717505275178488, Upper bound: 4.7200734584263975
ln(Common odds ratio): Lower bound: -0.39786824619789973, Upper bound: 1.5518243627022899
